In [ ]:
import os, numpy as np, sys, pandas as pd, matplotlib.pyplot as plt, stumpy, glob
from stumpy import config

# Import library with auxiliary codes
sys.path.append(os.path.join("..", "..", "lib"))
import essencials as es

In [ ]:
config.STUMPY_EXCL_ZONE_DENOM = 1

SOURCE_FOLDER = os.path.join('..', '..', 'Dataset')

MEASURE_FILE_PATH = 'VD_MEASURE_L0.CSV'
LABELED_FILE_PATH = 'VD_LABELED_L0.CSV'

SOURCE_FOLDER_LOCAL = os.path.join(SOURCE_FOLDER, 'DD-Local')
FOLDER_PATH = os.path.join(SOURCE_FOLDER_LOCAL, '**', MEASURE_FILE_PATH)
MAIN_LIST_local = sorted(glob.iglob(FOLDER_PATH, recursive=True ))

SOURCE_FOLDER_YT = os.path.join(SOURCE_FOLDER, 'YT-Online')
FOLDER_PATH = os.path.join(SOURCE_FOLDER_YT, '**', MEASURE_FILE_PATH)
MAIN_LIST_MEASURE_yt = sorted(glob.iglob(FOLDER_PATH, recursive=True ))

# Join the Two Lists
MAIN_LIST_MEASURE = MAIN_LIST_local + MAIN_LIST_MEASURE_yt

baseDir_seed = os.path.join('..', '..', 'Dataset', 'REF-Gold-Label')
FOLDER_PATH = os.path.join(baseDir_seed, '**', LABELED_FILE_PATH)
REFERENCE_LST = sorted(glob.iglob(FOLDER_PATH, recursive=True ))

## Select the time-serie

In [ ]:
path_file = REFERENCE_LST[0]
path_dir = os.path.dirname(path_file)
vd_labeled_path = os.path.join(path_dir, LABELED_FILE_PATH)
vd_labeled = pd.read_csv(vd_labeled_path)
vd_labeled.drop(columns=['Unnamed: 0'], inplace=True)

## Get the labeled frames.

In [ ]:
all_class = es.GET_ALL_CLASSES (vd_labeled)
print(all_class)

label_name = all_class[0]

reference_measures = es.GET_MEASURES_FROM_CLASS(vd_labeled, label_name)

frames = es.GET_FRAMES_FROM_CLASS (vd_labeled, label_name)


## Select the labeled subserie

In [ ]:
Q_df = vd_labeled.loc[frames]
Q_df = Q_df[reference_measures]

## Plot reference serie

In [ ]:
fonte = {'family': "Times New Roman", 'color': 'black', 'weight': 'bold', 'size': 10}
plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

plt.subplots(figsize=(10, 3))
plt.suptitle('Série Padrão (Q_df)', fontsize='10')
plt.xlabel('Time', fontsize ='10')
plt.ylabel('Amplitude', fontsize='10')
plt.plot(Q_df, label=Q_df.columns)
plt.legend()
plt.show()

## Plot dataset series

In [ ]:
VIDEO_ID = 2
current_path_location = MAIN_LIST_MEASURE[VIDEO_ID]
VD_MEASURE_DT = pd.read_csv(current_path_location)
VD_MEASURE_DT.drop(columns=['Unnamed: 0'], inplace=True)
T_df = VD_MEASURE_DT[reference_measures]

In [ ]:
MAIN_LIST_MEASURE

In [ ]:
plt.subplots(figsize=(10, 3))
plt.suptitle('Time-serie (T_df)', fontsize='10')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.plot(T_df,  linewidth=1)
plt.show()

## Busca os padrões na série do dataset

In [ ]:
c_list = []
for step in range(0,len(Q_df.columns)):
    comp_object = es.Comparing(Q_df[reference_measures[step]], T_df[reference_measures[step]])
    comp_object.calc_matches()
    comp_object.measure_name = reference_measures[step]
    c_list.append(comp_object)

## Plota todas as séries encontradas para cada medida

In [ ]:
fig, axs = plt.subplots(len(c_list), 1, figsize=(7, 5))
fig.subplots_adjust(left=0.1, bottom=0, right=0.9, top=0.95, wspace=0.1, hspace=0.1)

for i, comparing in enumerate(c_list):
    Q_z_norm = stumpy.core.z_norm(comparing.Q_df.values)

    axs[i].set_title(f'Raw results for {comparing.measure_name.upper()}', fontsize='10', loc='left', y=0.88, x=0.01)
    
    for match_distance, match_idx in comparing.matches_idxs:
        match_z_norm = stumpy.core.z_norm(comparing.T_df.values[match_idx:match_idx+len(comparing.Q_df)])
        axs[i].plot(match_z_norm, lw=0.5, label=f'Frame {match_idx}')
    axs[i].plot(Q_z_norm, lw=2, color="black", label="Seed")
    axs[i].legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='8')
    
fig.text(0.465, 0.025, 'Frame number', ha='center', fontdict=fonte)
fig.text(0.015, 0.5, 'Normalized Amplitude (pixels)', va='center', rotation='vertical', fontdict=fonte)

plt.tight_layout(pad=2.0, h_pad=1.0, w_pad=1.0)
plt.savefig('graph1.png')
plt.show()

## Seleciona apenas os index em matches_idxs, pois a estrutura tbm armazena a distancia de similaridade

In [ ]:
all_index = []
for c in c_list:  
    all_index.append(c.matches_idxs[:, 1])
    
threshold = 10

# filtra por coincidência a partir de um um treshold de distancia entre a posição dos indexs
aux = all_index.copy()

In [ ]:
print(aux)

filter_index = es.find_all_matches(aux, threshold)

filter_index_list=list(filter_index[0])
# Corrigir os index das subséries pelo index do frame original(frame_seq)
filter_index_begin = []
for idx_tuple in filter_index_list:
    filter_index_begin.append(idx_tuple)

idxs_match_frame_seq = list(VD_MEASURE_DT.loc[filter_index_begin, 'frame_seq'].values)

In [ ]:
idxs_match_frame_seq

## Recupera as infromações de cada indice a partir de matches_idxs e salva atualiza os objetos

## Plota as séries filtradas

In [ ]:
fig, axs = plt.subplots(len(c_list), 1, figsize=(7, 5))
fig.subplots_adjust(left=0.1, bottom=0, right=0.9, top=0.95, wspace=0.1, hspace=0.1)

for i, comparing in enumerate(c_list):
    Q_z_norm = stumpy.core.z_norm(comparing.Q_df.values)

    axs[i].set_title(f'Filtered results for {comparing.measure_name.upper()} (threshold = {threshold})', fontsize='10', 
                     loc='left', y=0.88, x=0.01)
    
    for match_distance, match_idx in comparing.filter_matches_idxs:
        match_z_norm = stumpy.core.z_norm(comparing.T_df.values[match_idx:match_idx+len(comparing.Q_df)])
        axs[i].plot(match_z_norm, lw=0.5, label=f'Frame {match_idx}')
    axs[i].plot(Q_z_norm, lw=2, color="black", label="Seed")
    axs[i].legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='8')

fig.text(0.465, 0.025, 'Frame number', ha='center', fontdict=fonte)
fig.text(0.015, 0.5, 'Normalized Amplitude (pixels)', va='center', rotation='vertical', fontdict=fonte)


plt.tight_layout(pad=2.0, h_pad=1.0, w_pad=1.0)
plt.savefig('graph2.png')
plt.show()

## Corrigir index por frame_seq

In [ ]:
path_dir = os.path.dirname(current_path_location)
vd_info_path = os.path.join(path_dir, 'VD_INFO.CSV')
VD_INFO_DT = pd.read_csv(vd_info_path)
VD_INFO_DT.drop(columns=['Unnamed: 0'], inplace=True)
path_video_file = '..\\Video-Source\\in_DD-Local\\' + VD_INFO_DT.link_video[0]+'.mp4'
frames = es.LOAD_VIDEO_FRAMES(path_video_file)

In [ ]:
start_frame = 681
end_frame = start_frame+len(Q_df)
n_frames_per_row = 5
es.DISPLAY_FRAMES(frames, start_frame, end_frame, n_frames_per_row)

In [ ]:
INDEX_DT = es.CREATE_FILES_INDEX(MAIN_LIST_MEASURE)

## Read or creates the VD_LABELED file if it does not exist.

In [ ]:
VIDEO_ID = 0
if INDEX_DT.loc[VIDEO_ID,'label_file_exist'] == 0:
    print('Creating Label File...')
    
    # First Initiate the labels = 0 means NO Label
    VD_MEASURE_DT['label_measures'] = str({})
    VD_LABELED_DT = VD_MEASURE_DT.copy()
else:
    VD_LABEL_FILE_NAME = 'VD_LABELED_LO.CSV'
    print('Reading Label File...') 
    VD_LABELED_DT = es.READ_CSV_FILE(INDEX_DT.loc[VIDEO_ID,'path'], VD_LABEL_FILE_NAME)

## Add information to label the frames.

In [ ]:
# You can copy the input parameters from the measure explorer json
# Type the Begin and End
label_name_insert = 'smile'
label_measure_insert = reference_measures
for label_idx in idxs_match_frame_seq:
    init_lab = label_idx
    end_lab = label_idx + len(Q_df)
    
    # Just Insert
    VD_LABELED_DT = es.UPDATE_LABEL_DF(init_lab, end_lab, label_name_insert, label_measure_insert, VD_LABELED_DT)

In [ ]:
idxs_match_frame_seq

In [ ]:
VD_LABELED_DT[idxs_match_frame_seq[0]:idxs_match_frame_seq[0]+len(Q_df)]